In [1]:
import IPython

def in_colab():
    try:
        import google.colab
        return True
    except:
        return False

SolutionToTask2 = lambda: IPython.display.display(IPython.display.Javascript('window.open("{url}");'.format(url='https://colab.research.google.com/drive/13OsSjKyzEyiyLdcKAuK6j_kn7aZ3yzxa?usp=sharing#scrollTo=Kaj-befXgS08' if in_colab() else 'QB41_A01_Classical_Gates_Solutions.ipynb#task2')))
SolutionToTask3 = lambda: IPython.display.display(IPython.display.Javascript('window.open("{url}");'.format(url='https://colab.research.google.com/drive/13OsSjKyzEyiyLdcKAuK6j_kn7aZ3yzxa?usp=sharing#scrollTo=Mh2tRWVhgS08' if in_colab() else 'QB41_A01_Classical_Gates_Solutions.ipynb#task3')))
SolutionToTask4 = lambda: IPython.display.display(IPython.display.Javascript('window.open("{url}");'.format(url='https://colab.research.google.com/drive/13OsSjKyzEyiyLdcKAuK6j_kn7aZ3yzxa?usp=sharing#scrollTo=bMYC0HmTgS09' if in_colab() else 'QB41_A01_Classical_Gates_Solutions.ipynb#task4')))
SolutionToTask5 = lambda: IPython.display.display(IPython.display.Javascript('window.open("{url}");'.format(url='https://colab.research.google.com/drive/13OsSjKyzEyiyLdcKAuK6j_kn7aZ3yzxa?usp=sharing#scrollTo=5umL4uZzgS09' if in_colab() else 'QB41_A01_Classical_Gates_Solutions.ipynb#task5')))
SolutionToTask6 = lambda: IPython.display.display(IPython.display.Javascript('window.open("{url}");'.format(url='https://colab.research.google.com/drive/13OsSjKyzEyiyLdcKAuK6j_kn7aZ3yzxa?usp=sharing#scrollTo=wsWwY8JZgS09' if in_colab() else 'QB41_A01_Classical_Gates_Solutions.ipynb#task6')))

if in_colab():
    !pip install qiskit[visualization]==0.43.3
    !pip install qiskit-aer

# Task 2 Odd to even


Design a quantum algorithm that when given numbers of range [1,n) and are odd
convert them into even numbers, and they must stay in the same range so they cannot be less than 1 nor greater than n. n = 2^k where k is the number of qubits you are going
to use.

In [9]:
# The code below was designed by Miguel Angel Luna Molinares for QOSF mentorship
# April, 2024

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange
from math import pi, sin, cos

#*******************************************************************************

# 1st step: Chosing a value for variable n

n = 16 # Upper constrain number, therefore our range would be [1,n)
       # You can change this variable to any number you want

print("Our range is going to be [1,",n,")")
print()

#*******************************************************************************

# 2nd step: Determing how many qubits we are going to use for

# In case we choose a number which are not correponding to 2 power to n (i.e. 2,4,8,16,32)
# we need to do some settings refering the number of qubits we will use
# If we choose 19 for variable n, we should use 5 qubits instead of 4 for using
# all numbers into the range [1,n) exclusively

if ((n**(1/2)-round(n**(1/2),0))>0) and ((n**(1/2)-round(n**(1/2),0))<1):
  N = int(round(n**(1/2),0) + 1) # Variable N would be the number of qubits
else:
   N = int(round(n**(1/2),0)) #Variable N would be the number of qubits

print("Number of qubits we will use would be N=",N," qubits") # Print number of qubits we will use
print()

#*******************************************************************************

# 3rd step: Creating a list of binary numbers from the range we choose

list_binary_numbers=[]
list_decimal_numbers=[]

for i in range(n):
  n_binary=bin(i).replace("0b", "") #Removing the characters 0b from the strings
  list_binary_numbers.append(n_binary)
  list_decimal_numbers.append(i)

print("Our list of decimal numbers corresponding to the range we previously choose")
print(list_decimal_numbers)
print()
print("Our list of binary numbers corresponding to the range we previously choose")
print(list_binary_numbers)
print()

# We need all decimal numbers have the same length of digits, the code below
# are going to fill the missing spots of 0's to the binary numbers which be needed

for i in range(n):
  if(len(list_binary_numbers[i])<N):
    zeros=''
    for j in range(N-len(list_binary_numbers[i])):
      zeros+='0'
    zeros+=list_binary_numbers[i]
    list_binary_numbers[i]=zeros

print("Our list of binary numbers all with the same lenght")
print(list_binary_numbers)
print()

#*******************************************************************************

# 4th step: Creating a list of random values starting from our list of binary
#           numbers we previously choose

list_random_binary_numbers = []

list_random_decimal_numbers=[]
for k in range(n):
  random_index=randrange(n)
  list_random_decimal_numbers.append(random_index)
  list_random_binary_numbers.append(list_binary_numbers[random_index])

print("Our list decimal numbers chosen randomly")
print(list_random_decimal_numbers)
print("Now the same list we just displayed before but in binary")
print(list_random_binary_numbers)
print()

#*******************************************************************************

# 5th step: Crearting the quantum circuits for our algorithm

counter=0
list_odd_binary_number = []

for input in list_random_binary_numbers:

  q_input = QuantumRegister(N,"Input register") # This quantum register corresponding to
                                                # inputs values (random number no matter it was even or not)
  q_output = QuantumRegister(N,"Output register") # This quantum register corresponding to
                                                  # output values (even number always)

  c = ClassicalRegister(N) # this classical register for measuremnts

  qc = QuantumCircuit(q_input,q_output,c) # a quantum circuit with quantum and classical registers

# We set up the input values

  j=N-1
  for i in range(N):
    if(input[j] == '1'):
      qc.x(i)
    j-=1
  qc.barrier()

# We perform some swap gates starting 2nd qubit to the last oone

  for i in range(2,N):
    qc.swap(q_input[i],q_output[i])
  qc.barrier()

# We perform two control-not gates and a toffoli gate, this allows to the algorithm to
# become a odd number into a even number

  qc.cx(q_input[0],q_output[1])
  qc.cx(q_input[1],q_output[1])
  qc.ccx(q_input[0],q_input[1],q_output[1])
  qc.barrier()

# We perform a measurement to all qubits

  qc.measure(q_output,c)

  job = execute(qc,Aer.get_backend('qasm_simulator'),shots=1)
  counts = job.result().get_counts(qc)

  list_odd_binary_number.append(str(counts).replace("{'", "").replace("': 1}", ""))

#*******************************************************************************

# 6th step: Showing results

  print()
  print()
  print("Random number=",int(list_random_binary_numbers[counter],2))
  print(qc)
  print()
  print("Showing the results")
  print("Before performing the algorithm ->",list_random_binary_numbers[counter])
  print("After performing the algorithm  ->",list_odd_binary_number[counter])
  print()

  counter+=1

list_odd_decimal_number = []
for i in range(n):
  list_odd_decimal_number.append(str(list_odd_binary_number[i]).replace("0b",""))
k=[]
for i in range(n):
  k.append(int(list_odd_decimal_number[i],2))

# Displaying final result of algorithm

print()
print("FINAL RESULTS")
print()
print("Original list of random numbers in binary")
print(list_random_binary_numbers)
print("Final list of only odd numbers in binary")
print(list_odd_binary_number)
print()
print("Original list of random numbers in decimal")
print(list_random_decimal_numbers)
print("Final list of only odd numbers in decimal")
print(k)
print()
print("Thanks for you attention!")


Our range is going to be [1, 16 )

Number of qubits we will use would be N= 4  qubits

Our list of decimal numbers corresponding to the range we previously choose
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Our list of binary numbers corresponding to the range we previously choose
['0', '1', '10', '11', '100', '101', '110', '111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']

Our list of binary numbers all with the same lenght
['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']

Our list decimal numbers chosen randomly
[4, 15, 5, 15, 11, 7, 14, 1, 3, 3, 12, 14, 1, 14, 2, 11]
Now the same list we just displayed before but in binary
['0100', '1111', '0101', '1111', '1011', '0111', '1110', '0001', '0011', '0011', '1100', '1110', '0001', '1110', '0010', '1011']



Random number= 4
                         ░        ░                 ░             
 Input register_0: ──────░────────░──